In [2]:
import asyncio, nest_asyncio, re, datetime
from pyppeteer import launch
import pandas as pd

nest_asyncio.apply()

In [4]:
def extract_ptcg_rarity_and_card_name(content):
    pattern = r'(UR|AR|SR|SAR)\s+[^\s\n]+'
    match = re.search(pattern, content)
    if match.group():
        return match.group().split(' ')[0], match.group().split(' ')[1]
    return None

def extract_ptcg_card_index(content):
    pattern = r'\d{3}/\d{3}'
    match = re.search(pattern, content)
    if match:
        return match.group()
    return None

def extract_card_price(content):
    pattern = r'\d{1,3}(?:,\d{3})* 円'
    match = re.search(pattern, content)
    if match:
        price_str = match.group()
        return int(price_str.replace(',', '').replace(' 円', ''))
    return None
    
async def extract_content(tcg_type, card_set, i):
    browser = await launch(headless=True)
    try:
        page = await browser.newPage()
        await page.goto(f'https://yuyu-tei.jp/sell/{tcg_type}/card/{card_set}/{i}', timeout=120000)
        await page.waitForSelector('.fw-bold', timeout=120000)
        print(f'https://yuyu-tei.jp/sell/{tcg_type}/card/{card_set}/{i}')
        fw_bold_texts = await page.evaluate('''() => {
            const boldElements = document.querySelectorAll('.fw-bold');
            return Array.from(boldElements).map(element => element.innerText).join('\\n');
        }''')
        return fw_bold_texts
    except Exception as e:
        print(f"Error for {card_set}/{i}: {e}")
        return None
    finally:
        await browser.close()
        
async def get_links(vers, rarity):
    browser = await launch(headless=True)
    try:
        page = await browser.newPage()
        await page.goto(f'https://yuyu-tei.jp/sell/poc/s/search?search_word={vers}&rare={rarity}&type=&kizu=0', timeout=120000)
        hyperlinks = await page.evaluate('''() => {
            const links = document.querySelectorAll('a');
            return Array.from(links).map(link => link.href);
        }''')
        return hyperlinks
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        await browser.close()


In [ ]:
links = []
for rarity in ptcg_rarity_table.keys():
    links += asyncio.run(get_links(''.join('&vers%5B%5D=' + i for i in ptcg_rarity_table[rarity]), rarity))
    
links = list(set([l for l in links if 'card' in l]))
sorted_links = sorted(links, key=lambda x: (x.split('/card/')[1].split('/')[0], int(x.split('/')[-1])))
len(sorted_links)

In [ ]:
sorted_links

In [ ]:
pkm_df = pd.DataFrame(columns=['card_set','card_rarity', 'card_name', 'card_index', 'card_price', 'created_time'])
idx=0

for link in sorted_links:
    idx += 1
    tcg_type = link.split('/')[-4]
    card_set = link.split('/')[-2]
    i = link.split('/')[-1]
    print(f'{idx}/{len(sorted_links)}')
    content = asyncio.run(extract_content(tcg_type, card_set, i))
    try:
        card_rarity, card_name = extract_ptcg_rarity_and_card_name(content)
    except:
        continue
    card_index = extract_ptcg_card_index(content)
    card_price = extract_card_price(content)
    created_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    pkm_df.loc[len(pkm_df)] = [card_set, card_rarity, card_name, card_index, card_price, created_time]

In [5]:
content = asyncio.run(extract_content('poc', 'svg', 10050))

https://yuyu-tei.jp/sell/poc/card/svg/10050


In [9]:
ptcg_rarity_table = {
    'S-TD': ['svg']
}
links = []
for rarity in ptcg_rarity_table.keys():
    links += asyncio.run(get_links(''.join('&vers%5B%5D=' + i for i in ptcg_rarity_table[rarity]), rarity))
    
links = list(set([l for l in links if 'card' in l]))
sorted_links = sorted(links, key=lambda x: (x.split('/card/')[1].split('/')[0], int(x.split('/')[-1])))
len(sorted_links)

47

In [7]:
card_rarity, card_name = extract_ptcg_rarity_and_card_name(content)

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
card_rarity

In [ ]:
card_name